In [43]:
using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs
all_nodes = collect(1:destination)
# last_node = maximum(edge)
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
function gx_bound(c_L, c_U, c, c_g, x_now, edge)
    
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f, "Current interdiction x = ", x_now)
    start_node = edge[:,1]
    end_node = edge[:,2]

    no_node = max(maximum(start_node), maximum(end_node) )
    no_link = length(start_node)


    function getShortestX(state, start_node, end_node, origin, destination)
        _x = zeros(Int, length(start_node))
        _path = enumerate_paths(state, destination)

        for i=1:length(_path)-1
            _start = _path[i]
            _end = _path[i+1]

            for j=1:length(start_node)
                if start_node[j]==_start && end_node[j]==_end
                _x[j] = 1
                break
                end
            end

        end
        _x
    end


    graph = Graph(no_node)
    distmx = Inf*ones(no_node, no_node)

    # Adding links to the graph
    for i=1:no_link
        add_edge!(graph, start_node[i], end_node[i])
        distmx[start_node[i], end_node[i]] = c_g[i]
    end

    # Run Dijkstra's Algorithm from the origin node to all nodes
    state = dijkstra_shortest_paths(graph, origin, distmx)
    label = state.dists
    pred = state.parents
    b_arc = ""
    
    for i = 1: length(state.parents)
        if state.parents[i] != 0 
            b_arc = string(b_arc, "(", state.parents[i], ",", i, ")")
        end
    end
    
    # Retrieving the shortest path
    path = enumerate_paths(state, destination)
    
    #parents = LightGraphs.DijkstraState(state, destination)

    # Retrieving the 'x' variable in a 0-1 vector
    y = getShortestX(state, start_node, end_node, origin, destination)
    #println(f,"y vector:", y)
    
    gx = sum(c_g[i]*y[i] for i = 1:no_link)    
    SP = sum(c[i]*y[i] for i = 1:length(c))
    T = Int64[]
    for i = 1:Len
        if pred[edge[i,2]] == edge[i,1]
            push!(T, 1)
        else
            push!(T, 0)
        end
    end

    y_index = findall(y .== 1)

#     println("Minimum Spanning Tree: ", T)
    #println("Shortest path y = ", y)
    #println("Indices of shortest path (edge) = ", y_index)
    #println("Nodes visited =", path)
    #println("Minimum Spanning Tree =", pred)
    #println("Node label =" ,label)

    return y, gx, SP
end




h1 = Model(() -> Gurobi.Optimizer(gurobi_env))
@variable(h1, 1 >= y_h[1:Len]>=0)
#@variable(h, q[1:Len]>=0)


#Setting constraint for start node

# println(leaving1)
leaving = findall(edge[:,1].== origin)
# leaving = 1 .* (edge[j=1:Len,1] .== origin)

@constraint(h1, sum(y_h[k] for k in leaving) == 1)
# print(1)
#Setting constraint for other nodes

for i in all_nodes
    if i != destination && i != origin
#             incoming = 1 .* (edge[j=1:Len,2] .== i)
#             leaving = -1 .* (edge[j=1:Len,1] .== i)
        incoming = findall(edge[:,2].== i)
        leaving = findall(edge[:,1].== i)
        @constraint(h1, sum(-y_h[k] for k in leaving) + sum(y_h[k] for k in incoming) == 0)
    end
end
function hx_bound(h, c_L, c_U, c, d, x_now, edge)
    
    c = (c_L + c_U)/2
    #println(f,"Current CELL's LB = ", c_L)
    #println(f,"Current CELL's UB = ", c_U)
    #println(f,"Current interdiction x = ", x_now)
    M = c_U - c_L

    h2 = copy(h1)

    y2 = h2[:y_h]

    @variable(h2, q[1:Len]>=0)
    for i = 1:Len
        @constraint(h2, q[i] >= c[i] - c_L[:,1][i] - M[i]*(1-y2[i])) #_h[i]))
    end

    @objective(h2, Min, sum((c_L[i]+d[i]*x_now[i])*y2[i] + q[i] for i=1:Len))

    #print(f,h2)
    set_optimizer(h2, ()-> Gurobi.Optimizer(gurobi_env))
    optimize!(h2)
#     println("h2 is fine")
    hx = JuMP.objective_value.(h2)
    
    return JuMP.value.(y2), hx
end

Academic license - for non-commercial use only


hx_bound (generic function with 1 method)

In [129]:
edge = [1 2; 1 3; 2 6; 3 4; 3 5; 4 6; 5 6; 1 7; 7 6]
c_L = [2.0, 1, 0, 0, 0, 12,30, 1, 6]
c_U = [2.0, 1,40, 0, 0, 12,60, 1, 12]
d = [100, 100, 0, 0, 0, 0, 0, 100, 0]

Len = length(edge[:,1])
b = 2

β = 0.8
K = 2
# p = ones(K)./K
Pset = [[1,3],[2,4,6],[8,9],[2,5,7]]
unc_arc = [3,7,9]
cset = []
push!(cset,[0  8;  0 60; 6 12]) #1
push!(cset,[8 40;  0 60; 6 12]) #2
# push!(cset,[ 0 10; 30 60; 6 7.5]) #3
# push!(cset,[20 40; 30 60; 6 7.5]) #4
# push!(cset,[ 0 10; 15 30; 6 7.5]) #5 
# push!(cset,[20 40; 15 30; 6 7.5]) #6
# push!(cset,[10 20;  0 7.5; 6 7.5]) #7
# push!(cset,[10 20; 30 60; 6 7.5]) #8
# push!(cset,[10 20; 15 30; 6 7.5]) #9

# push!(cset,[ 0 10;  0 7.5; 9 12]) #1
# push!(cset,[20 40;  0 7.5; 9 12]) #2
# push!(cset,[ 0 10; 30 60; 9 12]) #3
# push!(cset,[20 40; 30 60; 9 12]) #4
# push!(cset,[ 0 10; 15 30; 9 12]) #5 
# push!(cset,[20 40; 15 30; 9 12]) #6
# push!(cset,[10 20;  0 7.5; 9 12]) #7
# push!(cset,[10 20; 30 60; 9 12]) #8
# push!(cset,[10 20; 15 30; 9 12]) #9

# push!(cset,[ 0 10;  0 7.5; 7.5 9]) #1
# push!(cset,[20 40;  0 7.5; 7.5 9]) #2
# push!(cset,[ 0 10; 30 60; 7.5 9]) #3
# push!(cset,[20 40; 30 60; 7.5 9]) #4
# push!(cset,[ 0 10; 15 30; 7.5 9]) #5 
# push!(cset,[20 40; 15 30; 7.5 9]) #6
# push!(cset,[10 20;  0 7.5; 7.5 9]) #7
# push!(cset,[10 20; 30 60; 7.5 9]) #8
# push!(cset,[10 20; 15 30; 7.5 9]) #9

# push!(cset,[ 0 10; 7.5 11.25; 6 7.5]) #1
# push!(cset,[20 40; 7.5 11.25; 6 7.5]) #2
# push!(cset,[10 20; 7.5 11.25; 6 7.5]) #7
# push!(cset,[ 0 10; 7.5 11.25; 9 12]) #1
# push!(cset,[20 40; 7.5 11.25; 9 12]) #2
# push!(cset,[10 20; 7.5 11.25; 9 12]) #7
# push!(cset,[ 0 10; 7.5 11.25; 7.5 9]) #1
# push!(cset,[20 40; 7.5 11.25; 7.5 9]) #2
# push!(cset,[10 20; 7.5 11.25; 7.5 9]) #7

# push!(cset,[ 0 10; 11.25 15; 6 7.5]) #1
# push!(cset,[20 40; 11.25 15; 6 7.5]) #2
# push!(cset,[10 20; 11.25 15; 6 7.5]) #7
# push!(cset,[ 0 10; 11.25 15; 9 12]) #1
# push!(cset,[20 40; 11.25 15; 9 12]) #2
# push!(cset,[10 20; 11.25 15; 9 12]) #7
# push!(cset,[ 0 10; 11.25 15; 7.5 9]) #1
# push!(cset,[20 40; 11.25 15; 7.5 9]) #2
# push!(cset,[10 20; 11.25 15; 7.5 9]) #7

# println(cset)
p = zeros(K)
p = [0.2, 0.8]
# p = [0.0078125, 0.0078125, 0.03125, 0.0625, 0.03125, 0.03125, 0.0078125, 0.015625, 0.03125, 
#     0.015625, 0.015625, 0.0625, 0.125, 0.0625, 0.0625, 0.015625, 0.03125, 0.0625, 
#     0.0078125, 0.0078125, 0.03125, 0.0625, 0.03125, 0.03125, 0.0078125, 0.015625, 0.03125,
#     0.00390625, 0.00390625, 0.00390625, 0.0078125, 0.0078125, 0.0078125, 0.00390625, 0.00390625, 0.00390625,
#     0.00390625, 0.00390625, 0.00390625, 0.0078125, 0.0078125, 0.0078125, 0.00390625, 0.00390625, 0.00390625]
# p = [0.0625,0.0625,0.125, 0.25,0.125, 0.125,0.0625,0.0625,0.125,
#     0.0625,0.0625,0.125, 0.25,0.125, 0.125,0.0625,0.0625,0.125]./2
println(sum(p[i] for i = 1:length(p)))
# 0.0625,0.0625,0.0625, 0.25,0.125, 0.125,0.0625,0.0625,0.0625]
m = Model(() -> Gurobi.Optimizer(gurobi_env)) # If we want to add # in Gurobi, then we have to turn of Gurobi's own Cuts 
@variable(m, α)
@variable(m,  x[1:Len], Bin)
@variable(m, 1e6 >= z[1:K] >= 0)
@constraint(m, sum(x[i] for i=1:Len) <= b) #attack budget = 2

#Cell 1
for i = 1:K
#     println("k = ",i)
    for y in Pset
#         println(y)
        cbar_vector = cset[i]
#         println("cbar_vector = ", cbar_vector)
        SP = 0
        for arc in y
#             println("arc = ", arc)
            if (arc in unc_arc) == true
#                 println()
                arc_index_cset = findall(unc_arc.==arc)[1]
#                 println("arc_index_cset = ", arc_index_cset)
                SP = SP + (cbar_vector[arc_index_cset,1]+cbar_vector[arc_index_cset,2])/2
            else
                SP = SP + c_L[arc]
            end
        end
#         println("SP = ", SP)
        @constraint(m, α <= SP + sum(x[i]*d[i] for i in y) + z[i]) #Path 1-2-6
#         @constraint(m, α <= 13 + sum(x[i]*d[i] for i in [2,4,6]) + z[1]) #Path 1-3-4-6
#         @constraint(m, α <= 10 + sum(x[i]*d[i] for i in [8,9]) + z[1]) #Path 1-7-6
#         @constraint(m, α <= 8.5 + sum(x[i]*d[i] for i in [2,5,7]) + z[1]) #Path 1-3-5-6
    end
end


@objective(m, Max, α - (1/(1-β))*sum(p[i]*z[i] for i = 1:length(p)) )#w - sum(s[k] for k=1:length(s))/length(s) )
optimize!(m)
#     println("h2 is fine")
# println(m)
println(JuMP.objective_value.(m))
println("Interdiction ", findall(JuMP.value.(x).==1))
println("α = ",JuMP.value.(α) )
println("z = ",findall(JuMP.value.(z).>0 ))
println(p)
# println(d)
#Path 1-2-6
#Path 1-3-4-6
#Path 1-7-6
#Path 1-3-5-6
# println(sum(p[i] for i = 1:18))

1.0
13.0
Interdiction [1, 8]
α = 13.0
z = Int64[]
[0.2, 0.8]


In [125]:
println(p[28:36]./2)

[0.00390625, 0.00390625, 0.00390625, 0.0078125, 0.0078125, 0.0078125, 0.00390625, 0.00390625, 0.00390625]


In [121]:
edge = [1 2; 1 3; 2 6; 3 4; 3 5; 4 6; 5 6; 1 7; 7 6]

c_L = [2.0, 1,10, 0, 0, 12, 0, 1, 6]
c_U = [2.0, 1,20, 0, 0, 12,15, 1, 7.5]
# d[2] = 2
d = [100, 100, 0, 0, 0, 0, 0, 100, 0]
# println(myArray)
M = c_U - c_L
# a = 1
# println(M)
origin = 1
destination = 6


Len = length(d)
x_now = zeros(Len)

x_now[1] = 1
x_now[8] = 1

c = (c_L + c_U)/2
c_g = c + d.*x_now

y, gx, SP = gx_bound(c_L, c_U, c, c_g, x_now, edge)
println(findall(y.==1))
println(c_g[y.==1])
println("g = ", gx)
println(sum(y[i]*c[i] for i=1:length(d)))
yh, hx = hx_bound(h, c_L, c_U, c, d, x_now, edge)
println("h = ", hx)




[2, 5, 7]
[1.0, 0.0, 7.5]
g = 8.5
8.5
h = 7.0


In [ ]:
using JuMP 
using Gurobi
using LightGraphs
using DataFrames
using CSV
using TimerOutputs

edge = [1 2; 1 3; 2 3; 2 4; 3 4]
c_L =  [0.0, 8 , 11, 4 , 0]
c_U =  [0.0, 12, 11, 20, 0]
d =  [0.0, 1, 0, 8, 0]
Len = length(c_L)

c = 0.5*(c_L+c_U)
# c = [0, 10, 11, avg, 0]

#STARTING SOLUTION:
yy = zeros(Len)

# yy[1] = 1
# yy[4] = 1


SP_init = sum(yy[i]*c[i] for i = 1:Len)

p = [1.0]
g = [SP_init]
h = [0.0]

origin = 1
destination = 4

last_node = maximum(edge)
all_nodes = collect(1:last_node)

M = zeros(Len)

for i = 1:Len
    M[i] = c_U[i] - c_L[i]
end

case = 0
delta1 = 1
delta2 = 0.01

last_node = maximum(edge)

x_now = zeros(Len)
x_now[2]=1


gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)



#@objective(h, Min, sum((cL_orig[i]+d[i]*x_now[i])*y_h[i] + q[i] for i=1:Len))
#print(h1)
#solve(h1)

# print(x_now)

In [ ]:
c_g = c + d.*x_now

In [44]:

edge = [1 2; 1 3; 2 6; 3 4; 3 5; 4 6; 5 6; 1 7; 7 6;]

c_L = [2.0, 1, 0, 0, 0, 12, 0, 1, 6]
c_U = [2.0, 1,40, 0, 0, 12,60, 1, 12]
# d[2] = 2
d = [100, 100, 0, 0, 0, 0, 0, 100, 0]
# println(myArray)
M = c_U - c_L
# a = 1
# println(M)
origin = 1
destination = 6


Len = length(d)
x_now = zeros(Len)

x_now[1] = 1
x_now[8] = 1

c = (c_L + c_U)/2
c_g = c + d.*x_now
# #println(f,"Current CELL's LB = ", c_L)
# #println(f,"Current CELL's UB = ", c_U)
# #println(f,"Current interdiction x = ", x_now)
# M = c_U - c_L

h2 = copy(h1)

y2 = h2[:y_h]

@variable(h2, q[1:Len]>=0)
for i = 1:Len
    @constraint(h2, q[i] >= c[i] - c_L[:,1][i] - M[i]*(1-y2[i])) #_h[i]))
end

@objective(h2, Min, sum((c_L[i]+d[i]*x_now[i])*y2[i] + q[i] for i=1:Len))

# for i = 1:Len
#     temp = (c_L[i]+d[i]*x_now[i])
#     println(i, " : ", temp)
# end
# print(h2)
solve(h2)

hx = getobjectivevalue(h2)
# println("x_now = ", x_now)
y_g, gx, SP = gx_bound(c_L, c_U, c, c_g, x_now, edge)
# println("g = ", gx, "\ny = ", y_g, "\nSP = ", SP)
# println("y_h = ",getvalue(y2), " \nhx = ", hx)
# println("q = ", getvalue(q))
println("g = ", gx)
println("h = ", hx)

ErrorException: `solve` has been replaced by `optimize!`. Note that `solve` used to return a `Symbol` summarizing the solution while `optimize!` returns nothing and the status of the solution is queried using `termination_status`, `primal_status` and `dual_status`.

In [35]:
# include("myData9.jl")
# include("paper2Example.jl")
# edge = [1 2; 1 3; 2 3; 2 4; 3 4]

# cL_orig = [0,2.0,3, 5, 7]
# cU_orig = [0,4.0,3, 25, 7]
# # d[2] = 2
# d = [0, 1, 0, 1,0]

edge = [1 2; 1 3; 2 6; 3 4; 3 5; 4 6; 5 6; 1 7; 7 6;]

cL_orig = [2.0, 1, 0, 0, 0, 12, 0, 1, 6]
cU_orig = [2.0, 1,40, 0, 0, 12,60, 1, 12]
# d[2] = 2
d = [100, 100, 0, 0, 0, 0, 0, 100, 0]

# edge = [1 2; 1 3; 2 3]
# c_L =  [5.0,8,4]
# c_U =  [5.0,28,6]

# d = [0.0,100,1]
Len = length(d)
# # d = zeros(3)

# Len = length(cL_orig)

# c_orig = 0.5*(cL_orig+cU_orig)

# origin = 1
# destination = 3

# last_node = maximum(edge)
# all_nodes = collect(1:last_node)

# M_orig = zeros(Len)

# for i = 1:Len
#     M_orig[i] = cU_orig[i] - cL_orig[i]
# end

# last_node = maximum(edge)
# include("testing.jl")

# include("./Instances/testInstance_55_keep.jl")

# c_L = [31.0, 8.0, 33.0, 1.0, 28.0, 17.0, 8.5, 9.0, 5.0, 9.0, 10.0, 10.0, 20.0, 39.0, 19.0, 32.0, 28.0, 7.0, 20.0, 3.0, 13.0, 12.0, 24.0, 6.0]
# c_U = [31.0, 38.0, 33.0, 1.0, 28.0, 17.0, 16.0, 9.0, 5.0, 9.0, 10.0, 20.0, 20.0, 39.0, 19.0, 32.0, 28.0, 33.0, 20.0, 3.0, 17.5, 12.0, 24.0, 10.0]
# edge = [1 2; 1 5; 1 8; 2 3; 2 6; 
#     3 4; 3 6; 4 12;5 6;6 7;
#     6 10;7 10;7 12;8 9;9 10; 
#     10 11; 10 12; 11 12]
# c_L = [5,5,5,1,2.0,
#     2,0,2,2,1,
#     2.0,0,2.5,2.75,2,
#     2.0,1,0]
# c_U = [5,5,5,1,2.0,
#     2,0,2,2,1,
#     2.0,0,2.5,2.75,2,
#     2.0,1,0]
# cL_orig = copy(c_L)
# cU_orig = copy(c_U)
# d = zeros(length(c_L))
# unc = [3, 1, 11]

# c_L = copy(cL_orig) #[31.0, 8.0, 33.0, 1.0, 28.0, 17.0, 8.5, 9.0, 5.0, 9.0, 10.0, 10.0, 20.0, 39.0, 19.0, 32.0, 28.0, 7.0, 20.0, 3.0, 13.0, 12.0, 24.0, 6.0]
# c_U = copy(cU_orig) #[31.0, 38.0, 33.0, 1.0, 28.0, 17.0, 16.0, 9.0, 5.0, 9.0, 10.0, 20.0, 20.0, 39.0, 19.0, 32.0, 28.0, 33.0, 20.0, 3.0, 17.5, 12.0, 24.0, 10.0]

# myArray = Array{Int64,(0,length(d))}
y_temp = zeros(Int, length(d))
# println(y_temp)
myArray = y_temp' #[]
# println(myArray)
M_orig = cU_orig - cL_orig
M = copy(M_orig)
# a = 1
# println(M)
unc_arcs = [3,7,9]#findall(M.>0)
origin = 1
destination = 6
println("No. unc arcs = ", length(unc_arcs))
# for i in unc_arcs
#     a = a*(c_U[i]-c_L[i])/M[i]
# end
# println("p = ", a)

x_now = zeros(Int64, length(d))
x_now[1] = 1 #(1,2)
x_now[2] = 1 #(1,3)
# x_now[8] = 1 #(1,7)

# println(cL_orig)
# println(cU_orig)
# println(d)

β = collect(1:10)./10
for α in β
    K = 500000
    g = zeros(K)
    for i = 1:K
        #Temp
        c_L = copy(cL_orig) 
        c_U = copy(cU_orig)
    #     println(c_L)
        for j in unc_arcs
    #         println(j)
            c_L[j] = c_L[j] + rand() * M[j]

            c_U[j] = c_L[j]
    #         println("Arc ", j, " = ", c_L[j])
        end
    #     c_U = c_L
        c = c_L
        c_g = c + d.*x_now
#         println(c_L)
#         println(c_U)
#         println(c)
#         println(c_g)
#         println(x_now)
#         println(edge)
        #End of Temp
        y, gx, SP = gx_bound(c_L, c_U, c, c_g, x_now, edge)
#         println("1")
        g[i] = gx
#         println(findall(y.==1), " ", gx)
    #     y_ind = findall(y.>0)
    #     if (y in myArray) == false
    #     println(myArray)
    #     myArray = [myArray; y']
    #     println(myArray)
    #     myArray = unique(myArray, dims=1)
    #     end
    #     println(myArray)
    end

#     α = 0.25
    g = sort!(g)
    a = collect(1:Int64(K*α))
    # println(a)
#     println(g[length(a)])
    println("α = ", α, " : ", sum(g[i] for i in a)/length(a))
end

No. unc arcs = 3
α = 0.1 : 7.297230661388743
α = 0.2 : 7.599419344696768
α = 0.3 : 7.897328580617999
α = 0.4 : 8.195912338134669
α = 0.5 : 8.499951313972582
α = 0.6 : 8.799369920415975
α = 0.7 : 9.102675677592542
α = 0.8 : 9.397430580986951
α = 0.9 : 9.69857681179225
α = 1.0 : 10.003150043781192


In [ ]:
# myArray_uni = unique(myArray,1)
# for i = 1:length(myArray[:,1])
#     println(myArray[i,:])
# end
myArray = unique(myArray, dims=1)
# myArray = myArray[2:end, :]
println(myArray)
println("No. SP = ", length(myArray[:,1])-1)
println(rand(1:11,3))
println(rand(11)*50)
y_temp = myArray[3,:]
for i = 1 : length(y_temp)
    if y_temp[i] >0
        println(edge[i,:]," : ", c_L[i]) 
    end
end
println(edge[17,:])

In [ ]:
alpha = 0.3
x[1] -> 3.05
x[2] -> 3
alpha = 0.25
x[1] -> 2.5
x[2] -> 2.5

In [61]:
using JuMP
using DataFrames, Query

┌ Info: Precompiling Query [1a8c2f83-1ff3-5112-b086-8aa67b057ba1]
└ @ Base loading.jl:1192


In [4]:
a = collect(1:5)
println(a)
setdiff!(a,5)
setdiff!(a,1)
println(a)

[1, 2, 3, 4, 5]
[2, 3, 4]


In [114]:
y = [1,1,1,0]
yL = [0,1,1,0]
M = [3,3,3,2]
df_constraints = DataFrame(NUM = Int[],CELL = Int[], Y = Array[], SP = Float64[])
push!(df_constraints,(1,1,y,1))
push!(df_constraints,(2,2,yL,2))
push!(df_constraints,(3,1,[1,1,0,0],2))
M_path = broadcast(abs,y-yL).*M
println(broadcast(abs,y-yL))
println("M_path ", M_path)
S_k = findall(M_path.==maximum(M_path))

[1, 0, 0, 0]
M_path [3, 0, 0, 0]


1-element Array{Int64,1}:
 1

In [115]:
df_constraints

,NUM,CELL,Y,SP
,Int64,Int64,Array,Float64
1,1,1,"[1, 1, 1, 0]",1.0
2,2,2,"[0, 1, 1, 0]",2.0
3,3,1,"[1, 1, 0, 0]",2.0


In [124]:
arc_split = 2
q2 = df_constraints |> 
@filter(_.CELL == 1)|>
@filter(_.Y[arc_split] == 1) |>DataFrame
# @map({_.rownumber, _.name, _.children}) |> DataFrame
# println(q2.NUM)
# for i in eachrow(q2)
#    println(i.CELL)
# end

,NUM,CELL,Y,SP
,Int64,Int64,Array,Float64
1,1,1,"[1, 1, 1, 0]",1.0
2,3,1,"[1, 1, 0, 0]",2.0


In [125]:
for row in eachrow(q2)
    row.SP = row.SP + 1
end
q2

,NUM,CELL,Y,SP
,Int64,Int64,Array,Float64
1,1,1,"[1, 1, 1, 0]",2.0
2,3,1,"[1, 1, 0, 0]",3.0


In [111]:
df = DataFrame(name=["John", "Sally", "Kirk"], age=[23., 42., 59.], children=[3,5,2])

# x = df |>
#   @filter(_.age>20) |>
#     @filter(_.children > 2)|>
#   @map({_.DataFramesRows,_.name, _.children}) |>
#   DataFrame
println(nrow(df))

3


In [60]:
arc_split = 2
y = [i for i in eachrow(df_constraints) if (i.CELL .== 1) && (i.Y[arc_split] .== 1)]
println(y)

DataFrameRow{DataFrame,DataFrames.Index}[DataFrameRow
 Row │ CELL   Y             SP      
     │ Int64  Array         Float64 
─────┼──────────────────────────────
   1 │     1  [1, 1, 1, 0]      1.0, DataFrameRow
 Row │ CELL   Y             SP      
     │ Int64  Array         Float64 
─────┼──────────────────────────────
   3 │     1  [1, 1, 0, 0]      2.0]


In [37]:
y in t

true

In [132]:
K = [1,2,3]
K2 = [1,2]
println(K)
println(K2)
K2 = copy(K)
setdiff!(K2, 2)
println(K)
println(K2)

[1, 2, 3]
[1, 2]
[1, 2, 3]
[1, 3]


In [137]:
K = vcat(K,K2)

5-element Array{Int64,1}:
 1
 2
 3
 1
 3

In [138]:
K

5-element Array{Int64,1}:
 1
 2
 3
 1
 3

In [140]:
K2

2-element Array{Int64,1}:
 1
 3

In [152]:
setdiff!(K,K2)

1-element Array{Int64,1}:
 2

In [149]:
M = zeros(5)
M[2] = 1
println(M)

[0.0, 1.0, 0.0, 0.0, 0.0]


In [151]:
println(findall((M.==maximum(M)) .& (M.>0)))

[2]


In [154]:
using Polynomials
Polynomial([1,1])

Polynomial(1 + x)

In [155]:
a = 1
b = 2
println(a)
println(b)
a = b
a = 0
println(a)
println(b)


1
2
0
2


In [1]:
cL_orig = [15.0, 66.0, 75.0, 106.0, 157.0, 190.0, 31.0, 60.0, 7.0, 10.0, 48.0, 92.0, 155.0, 44.0, 65.0, 131.0, 7.0, 15.0, 58.0, 82.0, 96.0, 140.0, 28.0, 123.0, 38.0, 27.0, 68.0, 122.0, 69.0, 99.0, 45.0, 55.0, 110.0, 8.0, 25.0, 30.0, 71.0, 11.0, 70.0, 99.0, 91.0, 78.0, 55.0, 43.0, 51.0, 70.0, 86.0, 50.0, 29.0, 59.0, 72.0, 115.0, 73.0, 73.0, 59.0, 106.0, 66.0, 8.0, 11.0, 36.0, 127.0, 105.0, 70.0, 62.0, 36.0, 20.0, 29.0, 39.0, 151.0, 138.0, 126.0, 24.0, 5.0, 12.0, 52.0, 18.0, 13.0, 20.0, 128.0, 104.0, 38.0]
cU_orig = [15.0, 66.0, 75.0, 106.0, 157.0, 190.0, 31.0, 60.0, 7.0, 30.0, 48.0, 92.0, 155.0, 44.0, 65.0, 147.0, 7.0, 15.0, 74.0, 82.0, 96.0, 140.0, 28.0, 135.0, 54.0, 37.0, 68.0, 122.0, 69.0, 99.0, 45.0, 73.0, 110.0, 8.0, 25.0, 46.0, 71.0, 27.0, 70.0, 99.0, 91.0, 78.0, 55.0, 43.0, 51.0, 70.0, 86.0, 50.0, 29.0, 59.0, 72.0, 115.0, 85.0, 73.0, 59.0, 106.0, 66.0, 26.0, 11.0, 36.0, 127.0, 105.0, 88.0, 62.0, 54.0, 20.0, 29.0, 51.0, 151.0, 138.0, 126.0, 24.0, 5.0, 30.0, 52.0, 18.0, 13.0, 20.0, 138.0, 104.0, 38.0]

81-element Array{Float64,1}:
  15.0
  66.0
  75.0
 106.0
 157.0
 190.0
  31.0
  60.0
   7.0
  30.0
  48.0
  92.0
 155.0
   ⋮  
 138.0
 126.0
  24.0
   5.0
  30.0
  52.0
  18.0
  13.0
  20.0
 138.0
 104.0
  38.0

In [2]:
for i in [2, 30, 78]
   println(cL_orig[i], " ", cU_orig[i]) 
end

66.0 66.0
99.0 99.0
20.0 20.0


In [3]:
66+99+20

185